In [2]:
def readfile(file):
    with open(file) as f:
        contents = f.read()[:-1].split('\n\n')
    
    mapping_dictionary = {}

    seed_data = contents[0].split(": ")
    map_data = contents[1:]
    mapping_dictionary[seed_data[0]] = [int(i) for i in seed_data[1].split()]
    
    map_content_dictionary = {}
    for i in map_data:
        map_content_data = i.split(" map:\n")
        map_content_list = []
        for map in map_content_data[1].split("\n"):
            mapping_rule = {0:'destination_range_start',1:'source_range_start',2:'range_length'}
            mapping_content = {}
            for j in mapping_rule.keys():
                mapping_content[mapping_rule[j]] = int(map.split()[j])
            map_content_list.append(mapping_content)
        map_content_dictionary[map_content_data[0]] = map_content_list
    
    mapping_dictionary['mapping'] = map_content_dictionary

    return mapping_dictionary

def seed_mapper(identified_value, source, mapping):
    current_map_key = [i for i in mapping.keys() if i.split('-to-')[0]==source][0]
    active_map = mapping[current_map_key]

    new_identified_value = identified_value

    for map in active_map:
        if identified_value>=map['source_range_start'] and identified_value<map['source_range_start']+map['range_length']:
            new_identified_value = map['destination_range_start']+identified_value-map['source_range_start']
            break

    next_destination = current_map_key.split('-to-')[1]

    return new_identified_value if next_destination=='location' else seed_mapper(new_identified_value, next_destination, mapping)


def aoc_5a_solution(file):
    seed_and_mapping = readfile(file)
    location_list = [seed_mapper(seed,'seed',seed_and_mapping['mapping']) for seed in seed_and_mapping['seeds']]
    return min(location_list)

aoc_5a_solution('input.txt')

157211394

In [4]:
def inverse_seed_mapper(identified_value, source, mapping):
    current_map_key = [i for i in mapping.keys() if i.split('-to-')[1]==source][0]
    active_map = mapping[current_map_key]

    new_identified_value = identified_value

    for map in active_map:
        if identified_value>=map['destination_range_start'] and identified_value<map['destination_range_start']+map['range_length']:
            new_identified_value = map['source_range_start']+identified_value-map['destination_range_start']
            break

    next_destination = current_map_key.split('-to-')[0]
    return new_identified_value if next_destination=='seed' else inverse_seed_mapper(new_identified_value, next_destination, mapping)

def aoc_5b_solution(file):
    seed_and_mapping = readfile(file)
    seed_pair_list = [(seed_start, seed_and_mapping['seeds'][index+1]) for index, seed_start in enumerate(seed_and_mapping['seeds']) if index%2==0]
    mapping = seed_and_mapping['mapping']
    
    location_mapping_key = [i for i in mapping if i.split('-to-')[1]=='location'][0]
    location_mapping_ranges = mapping[location_mapping_key]
    identified_least_location_ceiling = [location['range_length'] for location in location_mapping_ranges if location['destination_range_start']==0][0]
    
    least_location_seed = ''
    for i in range(0,identified_least_location_ceiling):
        # marker to track the progress per 5 mil.
        if i%5000000==0 and i!=0:
            print(f'{i} data processed')
        seed_traced_from_least_location = inverse_seed_mapper(i,'location',mapping)
        identified_range = [j for j in seed_pair_list if j[0]<=seed_traced_from_least_location and j[0]+j[1]>seed_traced_from_least_location]
        if len(identified_range)!=0:
            least_location_seed=i
            break

    return least_location_seed

aoc_5b_solution('input.txt')

5000000 data processed
10000000 data processed
15000000 data processed
20000000 data processed
25000000 data processed
30000000 data processed
35000000 data processed
40000000 data processed
45000000 data processed
50000000 data processed


50855035